In [ ]:
# Authenticate to Earth Engine (run once)
import ee
ee.Authenticate()
ee.Initialize(project='openpas-463512')


In [ ]:
# Interactive exports: precipitation + Sentinel-2 indices (NDVI, NDMI, BSI, MSI) + Meta Global Canopy Height
# - Select variable, year (where applicable), and region(s)
# - Always uses native scale (S2 ~10 m, precip 1 km, canopy height ~1 m)
# - Exports to Google Drive (one file per region)

import datetime
import ee
import ipywidgets as widgets
from IPython.display import display, clear_output

PROJECT = 'openpas-463512'
ASSETS = {
    'aracena': 'projects/openpas-463512/assets/aracena',
    'geres': 'projects/openpas-463512/assets/geres',
    'ordesa': 'projects/openpas-463512/assets/ordesa',
    'quercy': 'projects/openpas-463512/assets/quercy',
    'zorita': 'projects/openpas-463512/assets/zorita',
}
DRIVE_FOLDER = 'GEE_OpenLandMap'

MONTHLY_BANDS = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

# ----------------------------- Helpers ------------------------------------

def mask_s2_sr(img: ee.Image) -> ee.Image:
    scl = img.select('SCL')
    mask = (scl.neq(3)
            .And(scl.neq(7))
            .And(scl.neq(8))
            .And(scl.neq(9))
            .And(scl.neq(10))
            .And(scl.neq(11)))
    return img.updateMask(mask)

def scale_sentinel(img: ee.Image) -> ee.Image:
    optical = img.select(['B2','B3','B4','B8','B11','B12']).divide(10000)
    return optical.addBands(img.select('SCL'))

def prep_s2(img: ee.Image) -> ee.Image:
    img = mask_s2_sr(img)
    scaled = scale_sentinel(img)
    # Preserve metadata including time_start
    scaled = scaled.copyProperties(img, img.propertyNames())
    scaled = scaled.set('system:time_start', img.get('system:time_start'))
    return scaled

def get_s2_collection(year: int, region: ee.Geometry) -> ee.ImageCollection:
    start = f"{year}-01-01"
    end = f"{year + 1}-01-01"
    col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate(start, end)
           .filterBounds(region)
           .map(prep_s2))
    return col

def compute_index(index: str, year: int, region: ee.Geometry) -> tuple[ee.Image, int, str]:
    col = get_s2_collection(year, region)
    if index == 'msi':
        col = col.filter(ee.Filter.calendarRange(6, 8, 'month'))
    composite = col.median()
    if index == 'ndvi':
        img = composite.normalizedDifference(['B8', 'B4']).rename('ndvi')
    elif index == 'ndmi':
        img = composite.normalizedDifference(['B8', 'B11']).rename('ndmi')
    elif index == 'bsi':
        swir = composite.select('B11')
        red = composite.select('B4')
        nir = composite.select('B8')
        blue = composite.select('B2')
        num = swir.add(red).subtract(nir.add(blue))
        den = swir.add(red).add(nir).add(blue).add(1e-6)
        img = num.divide(den).rename('bsi')
    elif index == 'msi':
        img = composite.select('B11').divide(composite.select('B8')).rename('msi')
    else:
        raise ValueError(f"Unsupported index: {index}")
    img = img.toFloat()
    try:
        crs = composite.select('B2').projection().crs().getInfo()
    except Exception:
        crs = 'EPSG:4326'
    return img.clip(region), 10, crs

def get_precip(region: ee.Geometry) -> tuple[ee.Image, int, str]:
    dataset = ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01')
    img = dataset.select(MONTHLY_BANDS).reduce(ee.Reducer.sum()).rename('annual_precip_mm').toInt16()
    crs = 'EPSG:4326'
    return img.clip(region), 1000, crs

def get_canopy_height(region: ee.Geometry) -> tuple[ee.Image, int, str]:
    col = ee.ImageCollection('projects/sat-io/open-datasets/facebook/meta-canopy-height')
    band = ee.String(col.first().bandNames().get(0))
    img = col.mosaic().rename('canopy_height_m').toFloat()
    try:
        crs = col.first().select([band]).projection().crs().getInfo()
        scale = col.first().select([band]).projection().nominalScale().getInfo()
    except Exception:
        crs = 'EPSG:4326'
        scale = 1
    return img.clip(region), scale, crs

def export_to_drive(image: ee.Image, region: ee.Geometry, scale: int, crs: str, name: str):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name[:100],
        folder=DRIVE_FOLDER,
        fileNamePrefix=name,
        region=region,
        scale=scale,
        crs=crs,
        fileFormat='GeoTIFF',
        maxPixels=1e13,
    )
    task.start()
    return task

def build_image(selection: str, year: int, region: ee.Geometry) -> tuple[ee.Image, int, str]:
    if selection == 'precipitation':
        return get_precip(region)
    if selection == 'canopy_height':
        return get_canopy_height(region)
    return compute_index(selection, year, region)

# ----------------------------- UI -----------------------------------------

variable_dropdown = widgets.Dropdown(
    options=[('Precipitation (annual)', 'precipitation'),
             ('NDVI (annual composite)', 'ndvi'),
             ('NDMI (annual composite)', 'ndmi'),
             ('BSI (annual composite)', 'bsi'),
             ('MSI (Jun-Aug)', 'msi'),
             ('Canopy Height (Meta 1m)', 'canopy_height')],
    value='ndvi',
    description='Variable:',
    style={'description_width': '130px'}
)

current_year = datetime.datetime.utcnow().year
year_dropdown = widgets.Dropdown(
    options=list(range(2017, current_year + 1)),
    value=current_year,
    description='Year:',
    style={'description_width': '130px'}
)

year_row = widgets.HBox([year_dropdown])

region_dropdown = widgets.Dropdown(
    options=['All regions'] + list(ASSETS.keys()),
    value='All regions',
    description='Region:',
    style={'description_width': '130px'}
)

run_button = widgets.Button(description='Start Drive export(s)', button_style='success')
out = widgets.Output()

def on_run_clicked(_):
    with out:
        clear_output()
        selection = variable_dropdown.value
        year = year_dropdown.value
        region_choice = region_dropdown.value
        targets = ASSETS.items() if region_choice == 'All regions' else [(region_choice, ASSETS[region_choice])]
        print(f"Variable: {selection} | Year: {year} | Regions: {[k for k,_ in targets]}")
        tasks = []
        for name, asset_id in targets:
            print(f"[INFO] Starting {name} ({asset_id}) ...")
            fc = ee.FeatureCollection(asset_id)
            region = fc.geometry()
            try:
                image, native_scale, crs = build_image(selection, year, region)
                year_tag = '' if selection in ('precipitation', 'canopy_height') else f"_{year}"
                prefix = f"{name}_{selection}{year_tag}"
                task = export_to_drive(image, region, native_scale, crs, prefix)
                tasks.append((name, task.id))
                print(f"   [OK] Export started. Task ID: {task.id}")
            except Exception as e:
                print(f"   [ERROR] {name}: {e}")
        print('Submitted tasks:', len(tasks))
        print('Monitor in https://code.earthengine.google.com/tasks or via ee.batch.Task.list()')

run_button.on_click(on_run_clicked)

ui = widgets.VBox([
    variable_dropdown,
    year_row,
    region_dropdown,
    run_button,
    out
])

display(ui)
